In [1]:
pip install selenium beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# creat service
service = Service(executable_path = "chromedriver.exe")
driver = webdriver.Chrome(service = service)
# open gg
driver.get("https://google.com")
#
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "gLFyf"))
)
# tuong tac voi thanh tim kiep google
# goi ham de tim phan tu bang cach su dung ten lop CLASS_NAME la "gLFyf" (ten lop cua thanh tim kiem gg) 
input_element = driver.find_element(By.CLASS_NAME, "gLFyf")
# dung send_keys de nhap tu khoa tim kiem va nhan tim kiem bang ENTER
input_element.send_keys("nguyenkim" + Keys.ENTER)
#click
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng"))
)

click_result = driver.find_element(By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng")
click_result.click()


#Truy cập vào 

tag = driver.find_elements(By.CLASS_NAME, 'nk-product-cate-homepage')
cl_tags = driver.find_elements(By.CLASS_NAME, 'nk-content')
link1 = []
link2 = []
time .sleep(15)

# Lấy các đường dẫn từ các thẻ cl_tags
for tag in cl_tags:
    try:
        link1.append(tag.find_element(By.TAG_NAME, 'a').get_attribute('href'))
    except:
       pass


# # In ra các đường dẫn
for l in link1:
    print(l)
# lấy danh sách





KeyboardInterrupt: 

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
# Tạo service
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)

# Mở Google
driver.get("https://google.com")

# Chờ cho thanh tìm kiếm xuất hiện
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "gLFyf"))
)

# Tương tác với thanh tìm kiếm
input_element = driver.find_element(By.CLASS_NAME, "gLFyf")
input_element.send_keys("nguyenkim" + Keys.ENTER)

# Chờ kết quả tìm kiếm và nhấp vào liên kết
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng"))
)
click_result = driver.find_element(By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng")
click_result.click()

# Chờ trang sản phẩm tải xong
time.sleep(5)  # Bạn có thể thay thế bằng WebDriverWait nếu cần


# Cuộn đến phần tử cần lấy dữ liệu
scroll = driver.find_element(By.CLASS_NAME,'nk-product-cate-homepage')
driver.execute_script("arguments[0].scrollIntoView();", scroll)
time.sleep(1)  # Đợi một chút trước khi cuộn đến phần tử tiếp theo
link1 = []
link2 =[]
# Truy cập vào thẻ để lấy link sản phẩm
cl_tags = driver.find_elements(By.CLASS_NAME, 'nk-content')
# Lấy các đường dẫn từ các thẻ cl_tags
for tag in cl_tags:
    try:
        # Thay đổi ở đây: dùng find_elements để lấy tất cả các thẻ <a>
        a_tags = tag.find_elements(By.TAG_NAME, 'a')
        
        # Lặp qua từng thẻ <a> và thêm vào danh sách
        for a_tag in a_tags:
            href = a_tag.get_attribute('href')
            if href:  # Kiểm tra nếu href không phải là None
                link1.append(href)
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

# In ra các đường dẫn sản phẩm gốc
for l in link1:
    print(l)




# Lấy danh sách sản phẩm từ trang đầu tiên
driver.get(link1[0])
time.sleep(10)


# Sử dụng XPATH để tìm tất cả các thẻ div có class chứa 'nk-product-cate-style-grid nk-product-collection'
product_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid') and contains(@class, 'nk-product-collection')]")

link2 = []

# Lặp qua các container sản phẩm và lấy đường dẫn từ thẻ <a> bên trong
for container in product_containers:
    try:
        # Tìm tất cả các thẻ <a> bên trong container
        a_tags = container.find_elements(By.TAG_NAME, 'a')

        # In ra số lượng thẻ <a> trong container để kiểm tra
        print(f"Số lượng thẻ <a> trong container: {len(a_tags)}")
        
        # Lặp qua từng thẻ <a> và thêm vào danh sách
        for a_tag in a_tags:
            href = a_tag.get_attribute('href')
            if href:  # Kiểm tra nếu href không phải là None
                link2.append(href)
                # In ra href để kiểm tra từng link
                print(f"Link thu được: {href}")
    except Exception as e:
        print(f"Lỗi khi lấy link từ container sản phẩm: {e}")

# In ra tất cả các đường dẫn sản phẩm đã thu thập được
print("Danh sách các đường dẫn sản phẩm thu thập được:")
for l2 in link2:
    print(l2)


# Tạo Danh sách để lưu dữ liệu các sản phẩm
# Tạo DataFrame
sp_df1 = pd.DataFrame(link1, columns=['Link Sản Phẩm'])
sp_df2 = pd.DataFrame(link2, columns=['Link Sản Phẩm Chi tiết'])

# Lưu vào file Excel với nhiều sheet
file_name = "spck4.xlsx"
with pd.ExcelWriter(file_name) as writer:
    sp_df1.to_excel(writer, sheet_name='Sản Phẩm', index=False)
    sp_df2.to_excel(writer, sheet_name='Chi Tiết Sản Phẩm', index=False)

print('Đã lưu thành công')
# Đóng trình duyệt
driver.quit()




https://www.nguyenkim.com/may-lanh/
https://www.nguyenkim.com/dien-thoai-di-dong/
https://www.nguyenkim.com/tu-lanh/
https://www.nguyenkim.com/tivi/
https://www.nguyenkim.com/may-tinh-xach-tay/
https://www.nguyenkim.com/may-giat/
https://www.nguyenkim.com/quat-vi/?features_hash=65-148987-148986-5954
https://www.nguyenkim.com/may-nuoc-nong/
https://www.nguyenkim.com/may-in-van-phong/
https://www.nguyenkim.com/noi-com-dien/
https://www.nguyenkim.com/noi-chien-khong-dau/
https://www.nguyenkim.com/may-loc-khong-khi/
https://www.nguyenkim.com/man-hinh-lcd-vi-tinh/
https://www.nguyenkim.com/may-ep-trai-cay/
https://www.nguyenkim.com/may-loc-nuoc/
https://www.nguyenkim.com/quat-vi/
https://www.nguyenkim.com/bep-tu-hong-ngoai
https://www.nguyenkim.com/amply-va-loa/
https://www.nguyenkim.com/may-tinh-bang/
https://www.nguyenkim.com/may-say-toc/
Số lượng thẻ <a> trong container: 120
Link thu được: https://www.nguyenkim.com/may-lanh-casper-inverter-tc-09is35.html
Link thu được: https://www.nguyen